In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
# 数据加载
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
# 数据探索
# 查看train_data信息
#pd.set_option('display.max_columns', None) #显示所有列
print('查看数据信息：列名、非空个数、类型等')
print(train_data.info())
print('-'*30)
print('查看数据摘要')
print(train_data.describe())
print('-'*30)
print('查看离散数据分布')
print(train_data.describe(include=['O']))
print('-'*30)
print('查看前5条数据')
print(train_data.head())
print('-'*30)
print('查看后5条数据')
print(train_data.tail())
# 使用平均年龄来填充年龄中的nan值
### inplace=True: 直接修改原对象
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)  
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
# 使用票价的均值填充票价中的nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

print(train_data['Embarked'].value_counts())
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features] ####读取多行
train_labels = train_data['Survived']
test_features = test_data[features]
print('特征值')
print(train_features)
dvec=DictVectorizer(sparse=False) ###sparse=False意思是不产生稀疏矩阵
## 转化为0，1向量
train_features=dvec.fit_transform(train_features.to_dict(orient='record'))
test_features=dvec.fit_transform(test_features.to_dict(orient='record'))
print(dvec.feature_names_)
print(train_features[1])
## 使用TPOT模型
#max_time_mins：How many minutes TPOT has to optimize the pipeline.
#max_eval_time_mins: How many minutes TPOT has to evaluate a single pipeline.
# verbosity: integer, optional (default=0)
# How much information TPOT communicates while it's running.
    # Possible inputs are:
        # 0, TPOT will print nothing,
        # 1, TPOT will print minimal information,
        # 2, TPOT will print more information and provide a progress bar, or
        # 3, TPOT will print everything and provide a progress bar.
        
##Please confirm that the input data is scikit-learn compatible. 
###For example, the features must be a 2-D array and target labels must be a 1-D array.



查看数据信息：列名、非空个数、类型等
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
------------------------------
查看数据摘要
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%

In [5]:
x_train, x_test, y_train, y_test = train_test_split(train_features, train_labels, train_size=0.75, test_size=0.25)

In [6]:
X_train.size, X_test.size, y_train.size, y_test.size

(6680, 2230, 668, 223)

In [7]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2, max_eval_time_mins=0.04, population_size=40)
tpot.fit(x_train, y_train)
tpot.score(x_test, y_test)

Generation 1 - Current best internal CV score: 0.83088317809449
Generation 2 - Current best internal CV score: 0.8323869374929863
Generation 3 - Current best internal CV score: 0.8324093816631131
Generation 4 - Current best internal CV score: 0.8398608461452138
Generation 5 - Current best internal CV score: 0.8398608461452138

2.01 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.25, min_samples_leaf=2, min_samples_split=2, n_estimators=100)


0.8385650224215246

In [8]:
#print(tpot.score(train_features, test_features))
tpot.export('tpot_mnist_pipeline.py')
Result=tpot.predict(test_features)
final = pd.DataFrame({'PassengerId': test_data['PassengerId'],'Sex': test_data['Sex'], 'Survived': Result})
print(final)

     PassengerId     Sex  Survived
0            892    male         0
1            893  female         0
2            894    male         0
3            895    male         0
4            896  female         1
..           ...     ...       ...
413         1305    male         0
414         1306  female         1
415         1307    male         0
416         1308    male         0
417         1309    male         1

[418 rows x 3 columns]
